In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

import pandas as pd
import numpy as np
import glob
import pickle
import random
import json
import gc
import os
import re

from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix

import warnings
warnings.simplefilter('ignore')

In [ ]:
def seed_everything(seed=42):
    print('Setting Random Seed')
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# Config

The weights used in this notebook (VERSION 5) are from a run with the best settings I managed to develop during the competition.

In [ ]:
PATH = "./"
SAVE_PATH = "./outputs/"

run_key = 'DAE_TRIAL'

CFG = {'debug': False,
       'debug_epochs': 2,
       'model_name': 'mlp',
       'lr' : 1e-04,
       'weight_decay': 9.72918866945795E-06,
       'epochs': 10,
       'device': 'cuda',
       'nfolds': 10,       
       'mlp_hidden_size': 391,
       'mlp_size_decline' : 1.0,
       'mlp_batch_size': 512,
       'mlp_dropout': 0.3,
       'bce_smooth' : 0.0001,
       'target_dae' : SAVE_PATH,
       'target_epoch' : f'{run_key}_model_checkpoint_600.pth',
       'random_state': 2021,
       'mlp_start_noise' : 0.15,
       'mlp_noise_decay' : 0.65,
       'run_key': run_key
       }

if CFG['debug']:
    CFG['epochs'] = CFG['debug_epochs']

with open(SAVE_PATH + f"{CFG['run_key']}_CFG.pickle", 'wb') as f:
    pickle.dump(CFG, f)
with open(SAVE_PATH + f"{CFG['run_key']}_CFG.json", 'w') as f:
    json.dump(CFG, f)

In [ ]:
#plot noise during MLP training
plt.plot(range(CFG['epochs']),
         np.array([CFG['mlp_start_noise']*(CFG['mlp_noise_decay']**e) for e in range(CFG['epochs'])]))
plt.xlabel('Epochs')
plt.ylabel('Noise During Training')

In [ ]:
dae_features = glob.glob(SAVE_PATH + f"{CFG['run_key']}_dae_features*.npy")
num_dae_features = np.load(dae_features[0]).shape[1]

In [ ]:
TARGET = 'service_canceled'

fts_continuous = ['customer_age_appl', 'time_start_process', 'operator_count', 'previous_customer_count']
fts_categorical = ['date', 'branch_name', 'customer_gender', 'customer_city', 'service_name_organization',
                   'service_name', 'service_name_2']

# unique counts should be the count of train PLUS test
# unique_counts = [4, 3, 3, 2, 2, 2, 334, 2, 408, 3, 726, 18, 12, 12, 13, 6, 31, 2]

print('Categorical Features', fts_categorical)
print('Continuous Features', fts_continuous)

print('Categorical Feature Count', len(fts_categorical))
print('Continuous Feature Count', len(fts_continuous))

In [ ]:
#this is just to control settings for the DAE hidden size etc, there is no further training of the DAE in this notebook

DAE_CFG = {'debug': False,
           'batch_size': 384, 
           'init_lr': 0.0003, 
           'lr_decay': 0.998,  # rate of decrease of learning rate
           'noise_decay': 0.999,  # rate of decrease of noise level
           'max_epochs': 2000, 
           'save_freq': 100, 
           'hidden_size': 1024,  # hidden_size == embed_dim * num_subspaces
           'num_subspaces': 8, 
           'embed_dim': 128, 
           'num_heads': 8, 
           'dropout': 0, 
           'feedforward_dim': 512, 
           'emphasis': 0.75, #weighing of loss to 'corrupted' data points - i tried varying over time, did not show immediate improvement
           'task_weights': [len(fts_categorical), len(fts_continuous)], #weighting for categorical vs continuous
           'mask_loss_weight': 2, #weighting of mask prediction vs prediction of reconstructed original data values
           'prob_categorical': 0.2, #probability of noise in categoricals
           'prob_continuous': 0.1, #probability of noise in continuous
           'random_state': 2021,
           'run_key': run_key
          }

model_params = dict(
    hidden_size=DAE_CFG['hidden_size'],
    num_subspaces=DAE_CFG['num_subspaces'],
    embed_dim=DAE_CFG['embed_dim'],
    num_heads=DAE_CFG['num_heads'],
    dropout=DAE_CFG['dropout'],
    feedforward_dim=DAE_CFG['feedforward_dim'],
    emphasis=DAE_CFG['emphasis'],
    mask_loss_weight=DAE_CFG['mask_loss_weight']
)

# DAE Code

In [ ]:
def feats_engineering(train, test):
    unique_counts = list()
    all_df = pd.concat([train, test]).reset_index(drop=True)
    
    all_df['customer_age_appl'].fillna(all_df['customer_age_appl'].mode()[0], inplace=True)
    all_df['time_start_process'].fillna(all_df['time_start_process'].mode()[0], inplace=True)

    all_df['customer_age_appl'] = all_df['customer_age_appl'].apply(lambda x: (int(x.split('-')[0])+int(x.split('-')[1]))/2)
    all_df['time_start_process'] = all_df['time_start_process'].apply(lambda x: int(x[:2]))

    for col in fts_categorical:
        unique_counts.append(all_df[col].nunique())
        
    df_train = all_df[:train.shape[0]]
    df_test = all_df[train.shape[0]:].reset_index(drop=True)
        
    return df_train, df_test, unique_counts

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import Dataset

def get_data():
    train = pd.read_csv(PATH + "queue_dataset_train_small_sample.csv")
    test = pd.read_csv(PATH + "queue_dataset_test.csv")
    
    train_data, test_data, unique_counts = feats_engineering(train, test)
    
    #combine train and test data vertically
    X_nums = np.vstack([
        train_data.loc[:, fts_continuous].to_numpy(),
        test_data.loc[:, fts_continuous].to_numpy()
    ])
    X_nums = (X_nums - X_nums.mean(0)) / X_nums.std(0) #normalize
    
    #stack the categorical data
    X_cat = np.vstack([
        train_data.loc[:, fts_categorical].to_numpy(),
        test_data.loc[:, fts_categorical].to_numpy()
    ])
    #encode the categoricals
    encoder = OneHotEncoder(sparse=False)
    X_cat = encoder.fit_transform(X_cat)
    
    #join the categorical and continuous data horizontally
    X = np.hstack([X_cat, X_nums])
    y = train_data[TARGET].to_numpy().reshape(-1, 1)
    
    return X, y, X_cat.shape[1], X_nums.shape[1], unique_counts #this lets us know how many categorical and continuous features there are


class FeatureDataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx], dtype=torch.float)            
        }
        return dct
    
    
class TestFeatureDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),   
        }
        return dct 

In [ ]:
bce_logits = torch.nn.functional.binary_cross_entropy_with_logits
mse = torch.nn.functional.mse_loss

class TransformerEncoder(torch.nn.Module):
    def __init__(self, embed_dim, num_heads, dropout, feedforward_dim):
        super().__init__()
        self.attn = torch.nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout)
        self.linear_1 = torch.nn.Linear(embed_dim, feedforward_dim)
        self.linear_2 = torch.nn.Linear(feedforward_dim, embed_dim)
        self.layernorm_1 = torch.nn.LayerNorm(embed_dim)
        self.layernorm_2 = torch.nn.LayerNorm(embed_dim)
    
    def forward(self, x_in):
        attn_out, _ = self.attn(x_in, x_in, x_in)
        x = self.layernorm_1(x_in + attn_out)
        ff_out = self.linear_2(torch.nn.functional.relu(self.linear_1(x)))
        x = self.layernorm_2(x + ff_out)
        return x


class TransformerAutoEncoder(torch.nn.Module):
    def __init__(
            self, 
            num_inputs, 
            n_cats, 
            n_nums, 
            hidden_size=1024, 
            num_subspaces=8,
            embed_dim=128, 
            num_heads=8, 
            dropout=0, 
            feedforward_dim=512, 
            emphasis=.75, 
            task_weights=[len(fts_categorical), len(fts_continuous)],
            mask_loss_weight=2,
        ):
        super().__init__()
        assert hidden_size == embed_dim * num_subspaces
        self.n_cats = n_cats
        self.n_nums = n_nums
        self.num_subspaces = num_subspaces
        self.num_heads = num_heads
        self.embed_dim = embed_dim
        self.emphasis = emphasis
        self.task_weights = np.array(task_weights) / sum(task_weights)
        self.mask_loss_weight = mask_loss_weight

        self.excite = torch.nn.Linear(in_features=num_inputs, out_features=hidden_size)
        self.encoder_1 = TransformerEncoder(embed_dim, num_heads, dropout, feedforward_dim)
        self.encoder_2 = TransformerEncoder(embed_dim, num_heads, dropout, feedforward_dim)
        self.encoder_3 = TransformerEncoder(embed_dim, num_heads, dropout, feedforward_dim)
        
        
        self.mask_predictor = torch.nn.Linear(in_features=hidden_size, out_features=num_inputs)
        self.reconstructor = torch.nn.Linear(in_features=hidden_size + num_inputs, out_features=num_inputs)

    def divide(self, x):
        batch_size = x.shape[0]
        x = x.reshape((batch_size, self.num_subspaces, self.embed_dim)).permute((1, 0, 2))
        return x

    def combine(self, x):
        batch_size = x.shape[1]
        x = x.permute((1, 0, 2)).reshape((batch_size, -1))
        return x

    def forward(self, x):
        x = torch.nn.functional.relu(self.excite(x))
        
        x = self.divide(x)
        x1 = self.encoder_1(x)
        x2 = self.encoder_2(x1)
        x3 = self.encoder_3(x2)
        x = self.combine(x3)
        
        predicted_mask = self.mask_predictor(x)
        reconstruction = self.reconstructor(torch.cat([x, predicted_mask], dim=1))
        return (x1, x2, x3), (reconstruction, predicted_mask)

    def split(self, t):
        return torch.split(t, [self.n_cats, self.n_nums], dim=1)

    #def feature(self, x):
        #attn_outs, _ = self.forward(x)
        #return torch.cat([self.combine(x) for x in attn_outs], dim=1)
    
    #i have modified the feature output to include the reconstruction / mask outputs
    #this needs checking in more detail - think range of values may be different
    def feature(self, x):
        #this returns the autoencoder layer outputs as a concatenated feature set
        attn_outs, _ = self.forward(x)
        attn_outs = torch.cat([self.combine(x) for x in attn_outs], dim=1)
        masks = torch.cat([x for x in _], dim=1)
        return torch.cat([attn_outs, masks], dim=1)

    def loss(self, x, y, mask, reduction='mean'):        
        _, (reconstruction, predicted_mask) = self.forward(x)
        
        x_cats, x_nums = self.split(reconstruction)
        y_cats, y_nums = self.split(y)
        w_cats, w_nums = self.split(mask * self.emphasis + (1 - mask) * (1 - self.emphasis))
        cat_loss = self.task_weights[0] * torch.mul(w_cats, bce_logits(x_cats, y_cats, reduction='none'))
        num_loss = self.task_weights[1] * torch.mul(w_nums, mse(x_nums, y_nums, reduction='none'))
        
        reconstruction_loss = torch.cat([cat_loss, num_loss], dim=1) if reduction == 'none' else cat_loss.mean() + num_loss.mean()
        
        mask_loss = self.mask_loss_weight * bce_logits(predicted_mask, mask, reduction=reduction)

        return reconstruction_loss + mask_loss if reduction == 'mean' else [reconstruction_loss, mask_loss]


In [ ]:
class SwapNoiseMasker(object):
    def __init__(self, probas):
        self.probas = torch.from_numpy(np.array(probas))

    def apply(self, X):
        #provides a distribution of points where we want to corrupt the data        
        should_swap = torch.bernoulli(self.probas.to(X.device) * torch.ones((X.shape)).to(X.device))
        
        #provides a corruped X output
        corrupted_X = torch.where(should_swap == 1, X[torch.randperm(X.shape[0])], X)
        
        #calculates the mask which we aim to predict
        mask = (corrupted_X != X).float()
        return corrupted_X, mask

# MLP Model

In [ ]:
class Model(nn.Module):
    def __init__(self, num_features=3000, num_targets=1, hidden_size=1000):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(CFG['mlp_dropout'])
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(CFG['mlp_dropout'])
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(CFG['mlp_dropout'])
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

# Smoothed BCE Loss

Note: I don't observe a huge benefit from smoothing - maybe a small gain from a very small smoothing amount. Just used this to have the option. There are some good discussions / notebooks on label smoothing elsewhere on Kaggle if not familiar.

In [ ]:
from torch.nn.modules.loss import _WeightedLoss

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)        
        
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

# Training Code

In [ ]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, epoch, device=CFG['device']):
    
    dae.eval()
    model.train()
    final_loss = 0  
    
    noise_maker = SwapNoiseMasker(CFG['mlp_start_noise']*(CFG['mlp_noise_decay']**epoch))
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        
        inputs, mask = noise_maker.apply(inputs)
        
        outputs = model(dae.feature(inputs)) #pass source data through DAE and MLP in one line
        loss = loss_fn(outputs, targets)
        loss.backward()
        
        optimizer.step()
        scheduler.step()        
        final_loss += loss.item()        
    final_loss /= len(dataloader)    
    return final_loss

def valid_fn(model, loss_fn, dataloader, device=CFG['device']):
    
    dae.eval()
    model.eval()
    final_loss = 0
    valid_preds = []    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(dae.feature(inputs))
        loss = loss_fn(outputs, targets)        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device=CFG['device']):
    
    dae.eval()
    model.eval()
    preds = []
    for data in dataloader:
        inputs = data['x'].to(device)
        with torch.no_grad():
            outputs = model(dae.feature(inputs))
        #this predicts by BATCH requiring listing and concatenation
        preds.append(outputs.sigmoid().detach().cpu().numpy())
    
    #then we need to concatenate the list of batches
    preds = np.concatenate(preds).reshape(-1,)  
    return preds


def run_training(X, y, len_train, len_test, folds, seed=42, batch_size=256, model_name='model',
                 num_features=3000,
                 num_targets=1,
                 hidden_size=1000,
                 device=CFG['device']):   
    
    print(len_train)
    print(len_train+len_test)
    
    seed_everything(seed)    
    
    #placeholder - out of fold predictions
    oof = np.zeros((len_train, ))
    
    #placeholder - test predictions
    predictions = np.zeros((len_test, ))
    
    #placeholder - training/validation graph
    fig,axes=plt.subplots(figsize=(18,6))
    
    #fold losses list
    fold_losses = []
    
    for fold in sorted(np.unique(folds,return_counts=False)): 
        train_idx=folds[:len_train]!=fold
        valid_idx=folds[:len_train]==fold
            
        print('     ')
        print(f'training for fold {fold}')
        #create the data set
        train_dataset = FeatureDataset(X[:len_train][train_idx], y[:len_train][train_idx])
        valid_dataset = FeatureDataset(X[:len_train][valid_idx], y[:len_train][valid_idx])

        #apply to the data loader
        trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
        validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)        
        
        #create the model itself
        model = Model(
            num_features=num_features,
            num_targets=num_targets,
            hidden_size=hidden_size,
        )

        #send to device and set up the loss and optimizer
        model.to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=CFG['lr'], weight_decay=CFG['weight_decay'],eps=0.00001 )
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                                  max_lr=1e-2, epochs=CFG['epochs'], steps_per_epoch=len(trainloader))

        loss_fn = nn.BCEWithLogitsLoss()
        loss_tr = SmoothBCEwLogits(smoothing =CFG['bce_smooth'])

        train_loss_list = []
        valid_loss_list = []
        best_loss = 9999999
        for epoch in range(CFG['epochs']):
            #the below updates the model and loss
            train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader,epoch, device)            
            train_loss_list+=[train_loss]
            
            #the below returns the validation predictions for the fold for each epoch
            valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, device)
            valid_loss_list+=[valid_loss]            
            
            print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}, valid_loss: {valid_loss}")
            
            if valid_loss < best_loss:
                #continue training if improving
                best_loss = valid_loss
                oof[valid_idx] = valid_preds.reshape(-1,)
                torch.save(model.state_dict(), SAVE_PATH + f"MODEL_{CFG['model_name']}_FOLD{fold}_SEED{seed}.pth")                    
                    
        fold_losses += [valid_loss_list[-1]]
                    
        del trainloader, validloader, train_dataset, valid_dataset
        gc.collect()

        sns.lineplot(x=range(CFG['epochs']), y=pd.Series(train_loss_list), color='Blue', ax=axes)
        sns.lineplot(x=range(CFG['epochs']), y=pd.Series(valid_loss_list), color='Red', ax=axes)
        
        #--------------------- PREDICTION---------------------
        #predict test data for fold
        testdataset = TestFeatureDataset(X[len_train:len_train+len_test])
        testloader = torch.utils.data.DataLoader(testdataset, batch_size=batch_size, shuffle=False)

        #we create the model and then we input the latest weights
        model = Model(
            num_features=num_features,
            num_targets=num_targets,
            hidden_size=hidden_size,
        )

        model.load_state_dict(torch.load(SAVE_PATH + f"MODEL_{CFG['model_name']}_FOLD{fold}_SEED{seed}.pth"))
        model.to(device)

        #predictions need to be added for the fold
        predictions += inference_fn(model, testloader, device)            
    
    print('finished with fold losses', fold_losses)
    
    predictions /= CFG['nfolds']
    return oof, predictions

# Load Data and Model and Create Folds

In [ ]:
#  get data
X, Y, n_cats, n_nums, unique_counts = get_data()

# setup model
dae = TransformerAutoEncoder(
    num_inputs=X.shape[1],
    n_cats=n_cats,
    n_nums=n_nums,
    **model_params
).cuda()

dae

In [ ]:
print('Using Simple Target Stratified KFold')
train = pd.read_csv(PATH + "queue_dataset_train_small_sample.csv")
test = pd.read_csv(PATH + "queue_dataset_test.csv")

skf = StratifiedKFold(n_splits=CFG['nfolds'], shuffle=True, random_state=CFG['random_state'])

folds=np.zeros((len(X),)).astype(np.int32)

for fold, (trn_idx, val_idx) in enumerate(skf.split(X[:train.shape[0]], Y)):
    folds[val_idx] = fold

fold_values = sorted(np.unique(folds, return_counts=False))
fold_values

# Load Weights and Train

In [ ]:
# load weights and train
model_state = torch.load(CFG['target_dae']+CFG['target_epoch'])
dae.load_state_dict(model_state['model'])

#note - I am not creating and saving features because the DAE will create the feature for each batch and immediately feed to MLP

#run training
# oof, predictions = run_training(X, Y.astype(np.long).reshape(-1), 
oof, predictions = run_training(X, Y, 
                                len_train=train.shape[0],
                                len_test=test.shape[0],
                                folds=folds,
                                seed=CFG['random_state'],
                                batch_size=CFG['mlp_batch_size'],
                                model_name='model',
                                num_features=num_dae_features+2*(np.array(unique_counts).sum()+len(fts_continuous)),
                                num_targets=1,
                                hidden_size=CFG['mlp_hidden_size'])

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import mode

pd.read_csv(PATH + "queue_dataset_train_small_sample.csv")

In [ ]:
#oof_df = pd.read_csv(PATH + 'train_data.csv')[['id', 'y']]
oof_df = train[['id', TARGET]]

oof_df['oof_prediction'] = np.where(oof>0.5, 1, 0)

acc_score = accuracy_score(oof_df[TARGET], oof_df['oof_prediction'])
print('CV Score', acc_score)

oof_df.to_csv(f"mlp_oof_cv{acc_score:.6f}.csv", index=False)
oof_df.head(10)

## Confusion matrix

In [ ]:
plt.figure(figsize=(16, 4))
cm = confusion_matrix(train[TARGET], np.where(oof>0.5, 1, 0))
sns.heatmap(cm, annot=True, fmt='3d')

## Submission

In [ ]:
submission = pd.read_csv(PATH + "queue_dataset_test.csv")
submission[TARGET] = np.where(predictions>0.5, 1, 0)
submission[['id', TARGET]].to_csv(f"mlp_submission_cv{acc_score:.6f}.csv", index=False)
submission.head(5)

In [ ]:
submission[TARGET].hist()